In [1]:
import pickle

with open('../../PNC_Good/MegaMeta3.pkl', 'rb') as f: 
    meta = pickle.load(f)
    
print('Complete')

Complete


In [2]:
with open('../../Work/LatentSim/Splits.pkl', 'rb') as f:
    keys, groups = pickle.load(f)
    
print(len(keys))
print(len(groups[0][0])+len(groups[0][1]))
print('Complete')

620
620
Complete


In [3]:
import numpy as np

nback = np.stack([meta[key]['nback'] for key in keys])
emoid = np.stack([meta[key]['emoid'] for key in keys])

print(nback.shape)
print(emoid.shape)

(620, 264, 231)
(620, 264, 210)


In [4]:
from nilearn.connectome import ConnectivityMeasure

def getFC(timeSeries, kind='correlation', transpose=True):
    connMeasure = ConnectivityMeasure(kind=kind)
    if transpose:
        timeSeries = np.transpose(timeSeries, axes=(0,2,1))
    return connMeasure.fit_transform(timeSeries)

nback_p = getFC(nback)
emoid_p = getFC(emoid)

print(nback_p.shape)

(620, 264, 264)


In [5]:
import torch

def convertTorch(p):
    t = torch.from_numpy(p).float()
    u = []
    for i in range(t.shape[0]):
        u.append(t[i][torch.triu_indices(264,264,offset=1).unbind()])
    return torch.stack(u).cuda()

nback_p_t = convertTorch(nback_p)
emoid_p_t = convertTorch(emoid_p)

print(nback_p_t.shape)
print(emoid_p_t.shape)
print('Complete')

torch.Size([620, 34716])
torch.Size([620, 34716])
Complete


In [6]:
mu_nback = torch.mean(nback_p_t, dim=0, keepdim=True)
mu_emoid = torch.mean(emoid_p_t, dim=0, keepdim=True)
std_nback = torch.std(nback_p_t, dim=0, keepdim=True)
std_emoid = torch.std(emoid_p_t, dim=0, keepdim=True)

nback_p_t = (nback_p_t - mu_nback)/std_nback
emoid_p_t = (emoid_p_t - mu_emoid)/std_emoid

print('Norm complete')

Norm complete


In [7]:
print(torch.min(nback_p_t[:,420:422],dim=0))
print(torch.min(emoid_p_t[:,420:422],dim=0))

torch.return_types.min(
values=tensor([-2.2998, -3.0133], device='cuda:0'),
indices=tensor([603, 350], device='cuda:0'))
torch.return_types.min(
values=tensor([-2.7284, -3.0616], device='cuda:0'),
indices=tensor([351,  33], device='cuda:0'))


In [8]:
age = np.stack([meta[key]['AgeInMonths'] for key in keys])
gen = np.stack([np.array([meta[key]['Gender'] == 'M', meta[key]['Gender'] == 'F']) for key in keys]).astype(int)
wrt = np.stack([meta[key]['wratStd'] for key in keys])

print(age.shape)
print(gen.shape)
print(wrt.shape)

(620,)
(620, 2)
(620,)


In [9]:
age_t = torch.from_numpy(age).float().cuda()
gen_t = torch.from_numpy(gen).float().cuda()
wrt_t = torch.from_numpy(wrt).float().cuda()

print('Complete')

Complete


In [65]:
import torch.nn as nn
import torch.nn.functional as F
import time

ceLoss = torch.nn.CrossEntropyLoss()
mseLoss = torch.nn.MSELoss()

rmse = []

def arith(n):
    return int(n*(n+1)/2)

def mask(e):
    return e - torch.diag(torch.diag(e.detach()))

def makeEdgeTarget(y, rng):
    e = torch.cdist(y.unsqueeze(1), y.unsqueeze(1))
    e[e == 0] = 1
    e = 1/e
    return e

class LatSim(nn.Module):
    def __init__(self, nTgts, inp, dp=0.5, edp=0.1, wInit=1e-4, dim=2, temp=1):
        super(LatSim, self).__init__()
        self.active = nn.Parameter(wInit*torch.randn(nTgts,inp.shape[-1],dim).float().cuda())
        self.dp = nn.Dropout(p=dp)
        self.edp = nn.Dropout(p=edp)
        self.temp = temp
    
    def getLatentsAndEdges(self, x, i):
        e = 1e-10
        y = torch.einsum('ac,ce->ae', x, self.active[i])
        e = e+y@y.T
        return y, y, e
        
    def forward(self, x, y, testIdcs=None):
        x = self.dp(x)
        res = []
        es = []
        for i in range(self.active.shape[0]):
            _, _, e = self.getLatentsAndEdges(x[:,i,:], i)
            if testIdcs is not None:
                e[:,testIdcs] = 0
            es.append(e.clone())
            e = self.edp(e)
            e = mask(e)
            e[e == 0] = float('-inf')
            e = F.softmax(e/self.temp, dim=1)
            res.append(e@y)
        return res, es

def validate(model, X, y, testIdcs):
    model.eval()
    with torch.no_grad():
        res, _ = model(X, y, testIdcs)
        avg = torch.mean(torch.stack(res), dim=0)
        if res[0].dim() == 1:
            loss = mseLoss(avg[testIdcs], y[testIdcs]).cpu().numpy()**0.5
        else:
            corr = (torch.argmax(avg, dim=1) == torch.argmax(y, dim=1))[testIdcs]
            loss = torch.sum(corr)/len(testIdcs)
    model.train()
    return loss, avg.detach().cpu().numpy()

for grp in range(1):
    trainIdcs = groups[grp][0][0:496]
    validIdcs = groups[grp][0]
    testIdcs = groups[grp][1]

    X0 = nback_p_t
    X1 = emoid_p_t
    Xreg = torch.stack([X0], dim=1)

    nEpochs = 1000
    pPeriod = 20
    thresh = 20

    sim = LatSim(1, Xreg, 0.5, 0.1, dim=20) # 0.2 wrat, 0.1 other
    optim = torch.optim.Adam(sim.parameters(), lr=1e-4, weight_decay=1e-4) 

    Xt = Xreg[trainIdcs]
    Xv = Xreg[validIdcs]

    trainLoss = []
    validLoss = []

    vIdcs1 = np.arange(496,len(validIdcs))
    vIdcs2 = validIdcs[496:]
    vIdcs3 = np.concatenate([testIdcs, vIdcs2])

    var = age_t
    etgt = makeEdgeTarget(var[trainIdcs], mult=6)

    for epoch in range(nEpochs):
        optim.zero_grad()
        res, es = sim(Xt, var[trainIdcs])
        avg = torch.mean(torch.stack(res), dim=0)
        loss = 0
        pairLoss = 0
        for r,e in zip(res + [avg], es + [es[0]]):
            loss += mseLoss(r, var[trainIdcs])
            pairLoss += 10*mseLoss(e, etgt)
        loss = torch.stack([loss, pairLoss])/(len(res)+1)
        torch.sum(loss).backward()
        optim.step()
        if epoch % pPeriod == 0 or epoch == nEpochs-1 or torch.all(loss[0:3] < thresh):
            print(f'epoch {epoch} loss={(float(loss[0]))} pairLoss={float(pairLoss)}')
            lossV, _ = validate(sim, Xv, var[validIdcs], vIdcs1)
            if len(validLoss) == 0 or lossV < min(validLoss):
                print(f'New best validation epoch {epoch} loss={lossV}')
                torch.save(sim.state_dict(), '../../Work/LatentSim/sim.pyt')
            validLoss.append(lossV)
            if torch.all(loss[0:3] < thresh):
                print('Early stopping')
                break

    print('Finished training')

    sim.load_state_dict(torch.load('../../Work/LatentSim/sim.pyt'))
    loss, avg = validate(sim, Xreg, var, testIdcs)
    rmse.append(loss)
    
#     yhat[testIdcs] = avg[testIdcs]
    print(f'FINISHED {grp} {loss}')

epoch 0 loss=1509.39990234375 pairLoss=149.38162231445312
New best validation epoch 0 loss=61.67731475830078
epoch 20 loss=2595.137939453125 pairLoss=1974.8973388671875
New best validation epoch 20 loss=43.525787353515625
epoch 40 loss=1277.4820556640625 pairLoss=192.2421112060547
New best validation epoch 40 loss=42.831233978271484
epoch 60 loss=684.9138793945312 pairLoss=173.40145874023438
New best validation epoch 60 loss=31.14763832092285
epoch 80 loss=400.3998718261719 pairLoss=165.91326904296875
New best validation epoch 80 loss=29.267398834228516
epoch 100 loss=241.87049865722656 pairLoss=163.2947540283203
New best validation epoch 100 loss=28.22511863708496
epoch 120 loss=146.8122100830078 pairLoss=161.85472106933594
New best validation epoch 120 loss=27.827346801757812
epoch 140 loss=108.25006103515625 pairLoss=161.46888732910156
New best validation epoch 140 loss=27.363027572631836
epoch 160 loss=81.6581039428711 pairLoss=161.38095092773438
New best validation epoch 160 loss=

In [67]:
print(e)

tensor([[ 2.5740, -1.2658, -1.4777,  ..., -0.7391, -0.5673, -2.1394],
        [-1.2658,  1.3484, -0.4048,  ..., -0.3890,  0.0381, -0.0918],
        [-1.4777, -0.4048,  2.7334,  ...,  1.8170,  0.8971,  3.1629],
        ...,
        [-0.7391, -0.3890,  1.8170,  ...,  1.5781,  0.9526,  2.1489],
        [-0.5673,  0.0381,  0.8971,  ...,  0.9526,  0.7275,  1.1542],
        [-2.1394, -0.0918,  3.1629,  ...,  2.1489,  1.1542,  3.9868]],
       device='cuda:0', grad_fn=<CloneBackward0>)
